<a href="https://colab.research.google.com/github/compi1234/pyspch/blob/master/test/spectrogram_test_ly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spectrogram Plotting with plotly

Using plotly as plotting backend has the advantage of creating an object in the browser that can be 
further manipulated by plotly-js, especially useful for zooming and scrolling.  The plots can be 
renedered inside a notebook or in a separate browser window.

Requirements:   
- plotly >= 4.12   
- pyspch

You may need to install following packages
> !pip install plotly=4.12   
> !pip install git+https://github.com/compi1234/pyspch.git     
> jupyter labextension install jupyterlab-plotly@4.14.3      (needed in Jupyter Lab, use appropriate version)

If in doubt on you local plotly setup, check:  https://plotly.com/python/getting-started/

In [1]:
# setting up for plotly plotting - you need to do this before importing pyspch.display
# use 'browser' as renderer to render in a separate browser window, otherwise the default is most likely OK
import os, sys
import numpy as np
import plotly
import plotly.io as pio
import plotly.graph_objects as go
os.environ['PYSPCH_BACKEND'] = 'plotly'
pio.renderers.default = "browser"
print("plotly: ",plotly.__version__," with renderer: ",pio.renderers.default)
####
from IPython.display import display, Audio, HTML, clear_output
import pyspch.spectrogram as specg
import pyspch.audio as audio
import pyspch.utils as spch
import pyspch.display as spch_disp
import pyspch.timit_file_tools as tio

plotly:  4.12.0  with renderer:  browser
pyspch(display): using PLOTLY backend !


In [3]:
# 1. set basename and directories
dir='https://homes.esat.kuleuven.be/~spchlab/data/'
file = "timit/si1027" #@param {type:"string"}
basename = dir+file
wavfname = basename + ".wav"
wrdfname = basename + ".wrd"
phnfname = basename + ".phn"
# 2. read the waveform data and compute a spectrogram
wavdata, sr = audio.load(wavfname)
f_shift = 0.01
spgdata = specg.spectrogram(wavdata,sample_rate=sr,f_shift=f_shift,n_mels=None)
(nparam,nfr) = spgdata.shape
# 3. get segmentations
segwrd = tio.read_seg_file(wrdfname,dt=1/sr,fmt='float32')
segphn = tio.read_seg_file(phnfname,dt=1/sr,fmt='float32',xlat='map61_cmu')

### 1. Elementary Waveform and Heatmap plotting using plotly

In [4]:
wave_go = go.Scatter(y=wavdata[0:16000], dx=1./sr,
                  showlegend=False,
                  hoverinfo="x+y",
                 )
fig_wav = go.Figure(wave_go)
fig_wav.show()

In [5]:
spg_go = go.Heatmap(z=spgdata,dx=f_shift,x0=f_shift/2.,dy=sr/(2*(nparam-1)),
                 colorscale='Jet',
                 showscale=False,
                 showlegend=False,
                 hoverinfo="x+y+z",
                 name='Spectrogram',
                 text=spgdata)
go.Figure(spg_go).show()

## 2. Plotting using the pyspch.display API's
In the example below we illustrate a multi-tier plot of speech sample, showing time aligned versions of sample waveform, spectrogram and varies (linguistic) segmentations

Everything is displayed using the plotly backend and making use of the RangeSlider for scrolling and zooming.
The RangeSlider copies one of the axis in which a selection is highlighted. The rest of the picture zooms in on the highlighted selection. 
However, be aware that the RangeSlider is displayed immediately below the axis that it is linked to, so the convenience solution is to organize the picture in such a way that the axis to be associated with the RangeSlider is at the bottom of the multi-axis plot.

In [6]:
# Multi-Segmentation Plot with RangeSlider
# with Rangeslider on Time Window
Rangeslider=True
Range_is_Time = True
if Range_is_Time: iwav = 3; ispg = 0; row_heights=[4,1,1,2];
else: iwav = 0; ispg = 3; row_heights=[2,1,1,4];
 
fig = spch_disp.make_rows(row_heights=row_heights,figsize=(20,12))
spch_disp.add_line_plot(fig,iwav,wavdata,dx=1./sr)
spch_disp.add_seg_plot(fig,1,segwrd,ypos=0.5)
spch_disp.add_seg_plot(fig,2,segphn,ypos=0.5,size=10,color="#000000")
spch_disp.add_img_plot(fig,ispg,spgdata,x0=f_shift/2.,dx=f_shift)
if(Rangeslider):
    fig['layout']['xaxis4'].update(title='Time(sec)',
                               type='linear',
                               rangeslider=dict(visible=True,borderwidth=2,bgcolor="#bbb"), 
                               showgrid=False)
fig.show()
#Audio(data=wavdata,rate=sr)

In [ ]:
# You can write out the plotly output to an HTML file, useful for interactive displays
fig.write_html("fig_plotly_1.html")